# О проекте


Задание относится к проекту UNICEF — международного подразделения ООН, чья миссия состоит в повышении уровня благополучия детей по всему миру. 


## Цель проекта

Отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска.

## Средства достижения целей

Реализовать достижение целей можно с помощью модели, которая предсказывала бы результаты госэкзамена по математике для каждого ученика школы.

## Задача проекта

Чтобы определиться с параметрами будущей модели, нужно провести разведывательный анализ данных и составьте отчёт по его результатам.

### Импорт библиотек

In [2]:
import warnings; warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import os
import re
import math

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from itertools import combinations

from scipy.stats import ttest_ind
from scipy.stats import norm
from scipy.stats import t
from statsmodels.stats import weightstats 


%matplotlib inline

### Инструменты для предобработки

352 µs ± 2.91 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Загрузка первичных данных

In [21]:
# 
incoming_files = 'stud_math.csv'

# 
if incoming_files not in os.listdir():
    print('\n!!! ВНИМАНИЕ !!!'*5)
    print('Не найден входной файл данных:', incoming_files)
else:
    print(r'[OK] Успешно пройдена проверка наличия входного файла датасета.')


[OK] Успешно пройдена проверка наличия входного файла датасета.


In [30]:
# 
stud = pd.read_csv(incoming_files)

# 
stud.shape

(395, 30)

In [31]:
# показывать больше строк
pd.set_option('display.max_rows', 50)

# показывать больше колонок
pd.set_option('display.max_columns', stud.shape[1] + 5) 

In [32]:
stud.sample(25)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
272,GP,F,18,U,LE3,T,1.0,1.0,other,other,home,mother,2.0,2.0,0.0,no,NaN,yes,no,no,-6.0,yes,no,no,4.0,4.0,3.0,3.0,2.0,55.0
267,GP,F,18,R,GT3,T,4.0,4.0,teacher,other,reputation,mother,2.0,2.0,0.0,no,NaN,yes,yes,yes,-6.0,yes,yes,no,4.0,3.0,4.0,4.0,8.0,55.0
184,GP,F,16,U,GT3,T,3.0,2.0,other,other,reputation,mother,1.0,2.0,0.0,no,yes,NaN,no,yes,-6.0,yes,yes,no,1.0,2.0,2.0,1.0,14.0,60.0
366,MS,M,18,U,LE3,T,4.0,4.0,teacher,services,other,mother,2.0,3.0,0.0,no,no,yes,no,yes,-9.0,yes,yes,yes,4.0,2.0,2.0,5.0,0.0,65.0
213,GP,M,18,U,GT3,T,2.0,2.0,services,other,home,mother,1.0,2.0,1.0,no,yes,yes,yes,yes,-6.0,yes,yes,no,4.0,4.0,4.0,5.0,15.0,40.0
329,GP,F,17,U,GT3,T,4.0,4.0,teacher,teacher,course,mother,2.0,3.0,0.0,no,yes,yes,no,no,-9.0,yes,yes,yes,4.0,3.0,3.0,4.0,4.0,70.0
130,GP,F,15,R,GT3,T,3.0,4.0,services,teacher,course,father,2.0,3.0,2.0,no,yes,no,no,yes,-9.0,yes,yes,yes,4.0,2.0,2.0,5.0,0.0,0.0
7,GP,F,17,U,GT3,A,4.0,4.0,other,teacher,home,mother,2.0,2.0,0.0,yes,yes,no,no,yes,-6.0,yes,no,no,4.0,1.0,4.0,1.0,6.0,30.0
1,GP,F,17,U,GT3,NaN,1.0,1.0,at_home,other,course,father,1.0,2.0,0.0,no,yes,no,no,no,-6.0,yes,yes,no,5.0,3.0,3.0,3.0,4.0,30.0
355,MS,F,18,U,GT3,T,3.0,3.0,services,services,course,father,1.0,2.0,0.0,no,yes,no,no,yes,-6.0,yes,no,yes,NaN,3.0,4.0,5.0,0.0,45.0


### Первичная обработка данных

### Распределение признака для числовых переменных

### Устранение выбросов

In [1800]:
# и убедиться что ни чего не пропустил)
len(answers)

27

### Оценка количества уникальных значений для номинативных переменных

### Преобразование данных

### Корреляционный анализ количественных элементов

### Отбор не коррелирующих переменных

### Анализ номинативных переменных, устранение не влияющих на предсказание конечной величины

In [1]:
# на переменную scope

### Выводы о качестве данных

### Выводы о данных, которые подходят для построения дальнейшей модели

### Спасибо за внимание!